In [18]:
import pandas as pd
import json

list = []

In [13]:
# https://www.fueleconomy.gov/feg/ws/index.shtml#ft4
# TO COME: https://ec.europa.eu/clima/news-your-voice/news/collecting-real-world-data-co2-emissions-and-fuel-consumption-new-cars-and-vans-2021-03-05_en
df = pd.read_csv("vehicles.csv", sep=',')

df = df.drop(['id'], axis=1).drop_duplicates()

for x, _df in df.groupby(by=["make", "year", "model", "eng_dscr", "engId", "cylinders", "trany", "drive"], dropna=False):
    
    consumer = {}
    consumer.update({'name': x[0] + " " + x[2] + " (" + str(x[1]) + ")"})
    consumer.update({'category': "car"})
    
    description = "Engine ID " + str(int(x[4])) 

    if not pd.isna(x[3]):
        description += " " + str(x[3]) 

    if x[5] != "" and pd.isna(x[5]) == False:
        description += ", " + str(int(x[5])) + " cylinders"

    if not pd.isna(x[6]):
        description += ", " + str(x[6])

    if not pd.isna(x[7]):
        description += ", " + str(x[7]) 

    consumer.update({'description': description})
    
    consumptions = {}
    
    for i, row in _df.iterrows():
        
         
        consumption = {}
        
        # purely electric
        if row['fuelType'] == "Electricity":
            value = row['combE']/160.9344
            consumption.update({'unit': "kWh"})

        # TODO: unclear how to handle CNG/LPG
        #else if row['fuelType'] == "Natural Gas":
        #    value = row['combE']/160.9344
        #    consumption.update({'unit': "kg"})

        else:
            value = 2.35215/row['comb08']
            consumption.update({'unit': "l"})

        consumption.update({'value': value})

        consumption.update({'base_unit': "km"})
        consumption.update({'reference_url': "https://www.fueleconomy.gov/"})
        
        consumptions[row['fuelType1'].lower()] = consumption
        
        # Hybrid
        if row['fuelType2'] == "Electricity":
            consumption = {}
            value = row['combE']/160.9344
            consumption.update({'unit': "kWh"})
            consumption.update({'value': value})
            consumption.update({'base_unit': "km"})
            consumption.update({'reference_url': "https://www.fueleconomy.gov/"})
        
            consumptions[row['fuelType2'].lower()] = consumption

            

    consumer.update({'consumptions': consumptions})
        
    list.append(consumer)   


In [19]:
df = pd.read_csv("meat.csv", sep=';')
df

,food,kg CO2e / kg
0,Andouille,16.20
1,Andouille de Guéméné,16.20
2,Andouille de Vire,16.20
3,Andouille/réchauffée à la poêle,16.20
4,Andouillette/crue,16.10
...,...,...
245,Saucisson sec aux noix et/ou noisettes,5.45
246,Saucisson sec pur porc,5.40
247,Saucisson sec pur porc/qualité supérieure,5.40
248,Terrine de canard,6.18


In [20]:
df = pd.read_csv("meat.csv", sep=';')


for i, row in df.iterrows():

    consumer = {}
    consumer.update({'name': row['food'] })
    consumer.update({'category': "food"})
    
    description = 'Number of CO2e kg per kg eaten'
    
    consumer.update({'description': description})
    
    consumption = {}
    
    consumption.update({'unit': "kg"})     
    consumption.update({'value': row['kg CO2e / kg']})
    consumption.update({'base_unit': "kg"})
    consumption.update({'reference_url': "https://base-empreinte.ademe.fr/donnees/jeu-donnees"})
    
    
    consumer.update({'consumptions': consumption})
    list.append(consumer)  

In [21]:
with open('consumers2.json', 'w', encoding='utf-8') as f:
    json.dump(list, f, ensure_ascii=False, indent=4)
